In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from new_service import get_information_eltiempo

def obtener_articulos(paginas=100):
    """
    Extrae los nombres de los artículos, URLs y fechas de publicación de las páginas especificadas
    de la búsqueda en El Tiempo sobre migrantes venezolanos en Colombia.

    :param paginas: Número de páginas a procesar.
    :return: DataFrame con las columnas 'Título', 'Fecha de Publicación' y 'URL'.
    """
    # Lista para almacenar los datos
    datos = []

    # Iterar sobre las páginas
    for page in range(paginas):
        url = f'https://www.eltiempo.com/buscar/?q=migrantes%20venezolanos%20en%20colombia&sort_field=_score&articleTypes=default,gallery,especial_modular,especial-tipo-d,video_detail,play_video_detail&categories_ids_or=&from=2024-01-20&until=2024-07-20&page={page}'
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Lanza un error para códigos de estado HTTP 4xx/5xx

        # Parsear el contenido HTML del artículo
        soup = BeautifulSoup(response.content, 'html.parser')
        articulos = soup.find_all('article', class_='c-article')

        # Extraer los nombres, URLs y fechas de publicación de los artículos
        for articulo in articulos:
            # Encontrar el título del artículo
            titulo = articulo.find('h3', class_='c-article__title').get_text(strip=True)
            # Encontrar la URL del artículo
            url = articulo.find('a', class_='page-link')['href']
            url = 'https://www.eltiempo.com/' + url
            # Encontrar la fecha de publicación
            fecha_publicacion = articulo.find('time', class_='c-article__date').get_text(strip=True)
            
            # Añadir los datos a la lista
            datos.append({
                'title': titulo,
                'date': fecha_publicacion,
                'url': url
            })

    # Crear un DataFrame con los datos
    df = pd.DataFrame(datos)
    return df

In [3]:
articles = obtener_articulos(paginas=100)

In [1]:
articles

NameError: name 'articles' is not defined

In [ ]:
articles['content'] = None
articles['#coments'] = None
articles['coments'] = None
articles['tags'] = None
for index, row in articles.iterrows():
    content, coments, tags = get_information_eltiempo(row['url'])
    # Agregar el contenido del articulo
    articles.loc[index, 'content'] = content
    if coments is not None:
        articles.loc[index, '#coments'] = len(coments)
        articles.loc[index, 'coments'] = coments
    if tags is not None:
        articles.loc[index, 'tags'] = tags

In [9]:
articles.to_excel('articulos_eltiempo.xlsx',index=False)